# Function to Identify fuel price cycles and various stats
short cycles work on peak to peak or trough to trough
long cycles incorporate a variable number of short cycles specified by the user

In [8]:
import pandas as pd
import os 
import plotly.graph_objects as go

# Set directory ### ADJUST WHEN DESIGNING FOR IMPLEMENTATION #### EXAMPLE SCRIPT PROTOTYPE ONLY ######
os.chdir(r'C:\Users\jaybl\Google Drive\Universidad\Current - Project 1\Fuel Price Repository - Group Only\Fuel-Price-Analysis\Model_Evaluation_And_Cycle_Tracker') 
print(os.getcwd())

# Load and view head of data
df = pd.read_csv('cycle_dummy_data_function_input___.csv')
subset = df['fuel_type'] == 'Unleaded 98' 
df = df[subset]
df = df.iloc[:, [0,2]]
df.head()

C:\Users\jaybl\Google Drive\Universidad\Current - Project 1\Fuel Price Repository - Group Only\Fuel-Price-Analysis\Model_Evaluation_And_Cycle_Tracker


,timestamp,target_value
4884,9/10/2016,128.8
4885,10/10/2016,127.8
4886,11/10/2016,126.7
4887,12/10/2016,128.9
4888,13/10/2016,136.4


In [9]:
def cycle_tracker(df, sensitivity, long_term):

    import pandas as pd
    import numpy as np

    # Takes an "ORDERED" series of dates and values from oldest to newest and outputs a dataframe of cycle stats 
    # date must be first column and value must be second column
    # sensitivity i.e. 0.05 = 5% for minimum change in peak to trough or trough to peak as additional criteria for cycle selection
    
    # re-name columns based on position because names not known
    df.rename(columns={ df.columns[0]: "date" }, inplace = True)
    df.rename(columns={ df.columns[1]: "value" }, inplace = True)
    
    # insert empty new columns
    
    # df['date'] col index 0
    # df['price'] col index 1
    
    # short term cycle
    df['movement'] = 0 # col index 2
    df['movement_perc'] = 0 # col index 3
    df['direction'] = 'steady' # col index 4
    df['peak_trough'] = 'mid-cycle' # col index 5
    df['peak_trough_strength_perc'] = 0 # col index 6
    df['absolute_change_since_last_peak_trough'] = 0 # col index 7
    df['peak_trough_days'] = 0 # col index 8
    df['full_cycle_number'] = 1 # col index 9
    df['full_cycle_days'] = 0 # col index 10
    df['full_cycle_current_strength'] = 0 # col index 11
    df['full_absolute_cycle_change'] = 0 # col index 12
    df['current_price_cycle_min_difference'] = 0 # col index 13
    df['current_price_cycle_max_difference'] = 0 # col index 14
    df['short_cycle_min'] = df.iloc[0, 1] # col index 15
    df['short_cycle_max'] = df.iloc[0, 1] # col index 16
    
    # long term cycle
    df['long_term_cycle_identifier'] = 1 # col index 17
    df['long_cycle_absolute_change'] = 0 # col index 18
    df['long_term_cycle_days'] = 0 # col index 19
    df['long_term_cycle_difference_min'] = 0 # col index 20
    df['long_term_cycle_difference_max'] = 0 # col index 21 
    df['long_term_cycle_min'] = df.iloc[0, 1]  # col index 22
    df['long_term_cycle_max'] = df.iloc[0, 1] # col index 23
    
    
    
    # for loop(1) set up 
    nrows = len(df)
    nrows1 =list(range(1, nrows))
    
    for index in nrows1:
    
        sub_index = index - 1
    
        # calculate movement values
        df.iloc[index, 2] = df.iloc[index, 1] - df.iloc[sub_index, 1] 
        df.iloc[index, 3] = (df.iloc[index, 1] - df.iloc[sub_index, 1])/df.iloc[index-1, 1]

        if df.iloc[index, 3] < 0:
            df.iloc[index, 4] = 'down'
        elif df.iloc[index, 3] > 0:
            df.iloc[index, 4] = 'up'
    
    ##########################################
    ##########################################
    ##########################################
    # Identify peaks and troughs
    ##################################
    
    # for loop(2) set up  
    start_index = min((df.direction.values == 'up').argmax(), (df.direction.values == 'down').argmax()) + 1
    nrows1 =list(range(start_index, nrows))
    last_value = df.iloc[start_index, 4]    
    cycle = 0
    
    for index in nrows1:
        
        sub_index = index - 1
    
        if (last_value == 'up') and (df.iloc[index, 4] == 'down') and (abs((df.iloc[sub_index, 1] - df.iloc[cycle, 1])/df.iloc[cycle, 1]) > sensitivity):
            df.iloc[sub_index, 5] = 'peak'
            last_value = 'down'
            cycle = sub_index

        elif (last_value == 'down') and (df.iloc[index, 4] == 'up') and (abs((df.iloc[sub_index, 1] - df.iloc[cycle, 1])/df.iloc[cycle, 1]) > sensitivity):
            df.iloc[sub_index, 5] = 'trough'
            last_value = 'up'
            cycle = sub_index
        
        elif (last_value == 'down') and (df.iloc[index, 4] == 'up') and (abs((df.iloc[sub_index, 1] - df.iloc[cycle, 1])/df.iloc[cycle, 1]) < sensitivity):
            last_value = 'up'
            cycle = sub_index
            
        elif (last_value == 'up') and (df.iloc[index, 4] == 'down') and (abs((df.iloc[sub_index, 1] - df.iloc[cycle, 1])/df.iloc[cycle, 1]) < sensitivity):
            last_value = 'down'
            cycle = sub_index
            
    #######################################        
    #######################################        
    #######################################
    ###### peak trough check ######
    start_index = min((df.peak_trough.values == 'trough').argmax(), (df.peak_trough.values == 'peak').argmax()) + 1
    nrows1 =list(range(start_index + 1, nrows))
    last_value = df.iloc[start_index, 5]
    min_point = 0
    min_index = 0
    max_point = 0
    max_index = 0
    
    # if 2 peaks or troughs in a row change them back to mid-cycle
        # when 2 of the same appear check that there is no opposite inbetween them. If there is rename it and this removes the double
            # if there is not then change the first of the double to mid cycle. The second is either lower or higher and supersedes it.
    # if there is no double pattern then move the index and move on        
    for index in nrows1:
        
        if df.iloc[index, 5] == 'trough' or df.iloc[index, 5] == 'peak' and df.iloc[index, 5] != last_value:
            last_value = df.iloc[index, 5]
            start_index = index
    
        elif df.iloc[index, 5] == 'trough' and df.iloc[index, 5] == last_value:
            max_index = df.iloc[start_index:index, 2].idxmax() # get max point index
            max_point = df.iloc[max_index, 2] # get max value
            
            if abs((max_point - df.iloc[start_index, 2])/max_point) > sensitivity:
                df.iloc[max_index, 5] = 'peak'
            else:
                df.iloc[start_index, 5] = 'mid-cycle'
        
        elif df.iloc[index, 5] == 'peak' and df.iloc[index, 5] == last_value:
            min_index = df.iloc[start_index:index, 2].idxmin() # get min point index
            min_point = df.iloc[min_index, 2] # get min value
            
            if abs((min_point - df.iloc[start_index, 2])/min_point) > sensitivity:
                df.iloc[min_index, 5] = 'trough'
            else:
                df.iloc[start_index, 5] = 'mid-cycle'
                
    ####################################### 
    #######################################
    #######################################

    # for loop(3) set up
    nrows1 =list(range(1, nrows))
    sub_index = 0
    
    # calculate movements and number of days from last peak or trough to current date (full peak trough movement) 
    for index in nrows1:
    
        if  df.iloc[index, 5] == 'trough' or df.iloc[index, 5] == 'peak':
            df.iloc[index, 6] = (df.iloc[index, 1] - df.iloc[sub_index, 1])/df.iloc[sub_index, 1]
            df.iloc[index, 7] = df.iloc[index, 1] - df.iloc[sub_index, 1]
            df.iloc[index, 8] = index - sub_index
            sub_index = index
        else:
            df.iloc[index, 6] = (df.iloc[index, 1] - df.iloc[sub_index, 1])/df.iloc[sub_index, 1]
            df.iloc[index, 7] = df.iloc[index, 1] - df.iloc[sub_index, 1]
            df.iloc[index, 8] = index - sub_index
            
    # for loop(4) set up
    cycle_point = min((df.peak_trough.values == 'peak').argmax(), (df.peak_trough.values == 'trough').argmax()) 
    start_cycle_point = df.iloc[cycle_point, 5] # first peak/trough value
    nrows1 =list(range(0, nrows))
    cycle = 1
    sub_index = 0
    
    # Identify all cycles - based on trough to trough or peak to peak. Whichever occurs first in the series
    for index in nrows1:
    
        if df.iloc[index, 5] == start_cycle_point:
            df.iloc[index, 9] = cycle
            df.iloc[index, 10] = index - sub_index
            cycle += 1
            sub_index = index
        else:
            df.iloc[index, 9] = cycle
            df.iloc[index, 10] = index - sub_index
            
    # for loop(5) set up
    nrows1 =list(range(1, nrows - 1))
    cycle_index = 0
    
    # calculate full cycle movements from last day of the last cycle to the current day up to the last day of the current cycle 
    for index in nrows1:
    
        sub_index = index + 1
    
        if df.iloc[index, 9] != df.iloc[sub_index, 9]:
            df.iloc[index, 11] = (df.iloc[index, 1] - df.iloc[cycle_index, 1])/df.iloc[cycle_index, 1]
            df.iloc[index, 12] = df.iloc[index, 1] - df.iloc[cycle_index, 1]
            cycle_index = index # last day of the last cycle
        
        else:
            df.iloc[index, 11] = (df.iloc[index, 1] - df.iloc[cycle_index, 1])/df.iloc[cycle_index, 1]
            df.iloc[index, 12] = df.iloc[index, 1] - df.iloc[cycle_index, 1]
     
    df.iloc[index + 1, 11] = (df.iloc[index + 1, 1] - df.iloc[cycle_index, 1])/df.iloc[cycle_index, 1]
    df.iloc[index + 1, 12] = df.iloc[index + 1, 1] - df.iloc[cycle_index, 1]
    
    
  
 # calculate current price difference from current full cycle min and max 
 # setup for loop(6) set up
    nrows1 =list(range(1, nrows))
    cycle_min = df.iloc[0, 1]
    cycle_max = df.iloc[0, 1]

   
    # cycle start is considered last day of the previous cycle
    for index in nrows1:
    
        sub_index = index -1
    
        if (df.iloc[index, 9] == df.iloc[sub_index, 9]) & (cycle_max < df.iloc[index, 1]):
            cycle_max = df.iloc[index, 1]
    
        elif (df.iloc[index, 9] == df.iloc[sub_index, 9]) & (cycle_min > df.iloc[index, 1]):
            cycle_min = df.iloc[index, 1]
   
        elif df.iloc[index, 9] != df.iloc[sub_index, 9]:
            cycle_min = df.iloc[index, 1]
            cycle_max = df.iloc[index, 1]
                
        df.iloc[index, 13] = df.iloc[index, 1] - cycle_min           
        df.iloc[index, 14] = df.iloc[index, 1] - cycle_max
        
        
    # Identify short term cycle min and max
    
    # for loop(7) set up
    nrows1 =list(range(1, nrows))
    cycle = 1
    sub_index = 0
    
    for index in nrows1:
        
        if df.iloc[index, 9] == cycle:
            df.iloc[index, 15] = min(df.iloc[sub_index:index + 1, 1])
            df.iloc[index, 16] = max(df.iloc[sub_index:index + 1, 1])
            
        elif df.iloc[index, 9] != cycle:
            sub_index = index
            df.iloc[index, 15] = df.iloc[index, 1]
            df.iloc[index, 16] = df.iloc[index, 1]
            cycle += 1
    
    ###### Long term cycle identification
    
    cycles = df.iloc[:, 9]/long_term
    long_cycles = cycles.apply(np.floor) + 1
    long_cycles = long_cycles.astype(int)
    df['long_term_cycle_identifier'] = long_cycles
    
    # Identify absolute change in long term cycle
    
    # for loop(8) set up
    nrows1 =list(range(1, nrows - 1))
    cycle_index = 0
    
    # calculate full long cycle movements from last day of the last cycle to the current day up to the last day of the current cycle 
    for index in nrows1:
    
        sub_index = index + 1
    
        if df.iloc[index, 17] != df.iloc[sub_index, 17]:
            df.iloc[index, 18] = df.iloc[index, 1] - df.iloc[cycle_index, 1]
            cycle_index = index # last day of the last cycle
        
        else:
            df.iloc[index, 18] = df.iloc[index, 1] - df.iloc[cycle_index, 1]
     
    df.iloc[index + 1, 18] = df.iloc[index + 1, 1] - df.iloc[cycle_index, 1]
    
    
    # identify number of days in long term cycle
    
    # for loop(9) set up
    nrows1 =list(range(0, nrows))
    cycle = 1
    sub_index = 0
    
    for index in nrows1:
    
        if df.iloc[index, 17] == cycle:
            df.iloc[index, 19] = index - sub_index
            
        elif df.iloc[index, 17] != cycle:
            sub_index = index
            df.iloc[index, 19] = index - sub_index
            cycle += 1
            
            
    # Identify difference from max and min values in long term cycle
    
    # for loop(10) set up
    nrows1 =list(range(1, nrows))
    cycle_min = df.iloc[0, 1]
    cycle_max = df.iloc[0, 1]
    
    # cycle start is considered last day of the previous cycle
    for index in nrows1:
    
        sub_index = index -1
    
        if (df.iloc[index, 17] == df.iloc[sub_index, 17]) & (cycle_max < df.iloc[index, 1]):
            cycle_max = df.iloc[index, 1]
    
        elif (df.iloc[index, 17] == df.iloc[sub_index, 17]) & (cycle_min > df.iloc[index, 1]):
            cycle_min = df.iloc[index, 1]
   
        elif df.iloc[index, 17] != df.iloc[sub_index, 17]:
            cycle_min = df.iloc[index, 1]
            cycle_max = df.iloc[index, 1]
                
        df.iloc[index, 20] = df.iloc[index, 1] - cycle_min           
        df.iloc[index, 21] = df.iloc[index, 1] - cycle_max
        

    # Identify long term cycle min and max
    
    # for loop(11) set up
    nrows1 =list(range(1, nrows))
    cycle = 1
    sub_index = 0
    
    for index in nrows1:
        
        if df.iloc[index, 17] == cycle:
            df.iloc[index, 22] = min(df.iloc[sub_index:index +1, 1])
            df.iloc[index, 23] = max(df.iloc[sub_index:index +1, 1])
            
        elif df.iloc[index, 17] != cycle:
            sub_index = index
            df.iloc[index, 22] = df.iloc[index, 1]
            df.iloc[index, 23] = df.iloc[index, 1]
            cycle += 1
    
            
    ###### Create short cycle stats dataframe ######
    cycle_stats_short = df.groupby("full_cycle_number")['value'].describe().reset_index()
    cycle_stats_short.rename(columns={ 'count': 'number_days'}, inplace = True)

    nrows = len(cycle_stats_short)
    nrows1 =list(range(0, nrows))
            
    cycle_stats_short['range_width'] = 0            

    for index in nrows1:
        cycle_stats_short.iloc[index, 9] = cycle_stats_short.iloc[index, 8] - cycle_stats_short.iloc[index, 4]
        
        
    ###### Create long cycle stats dataframe ######
    cycle_stats_long = df.groupby("long_term_cycle_identifier")['value'].describe().reset_index()
    cycle_stats_long.rename(columns={ 'count': 'number_days'}, inplace = True)

    nrows = len(cycle_stats_long)
    nrows1 =list(range(0, nrows))
            
    cycle_stats_long['range_width'] = 0            

    for index in nrows1:
        cycle_stats_long.iloc[index, 9] = cycle_stats_long.iloc[index, 8] - cycle_stats_long.iloc[index, 4]
    
    #### return both datasets #####
    return(df, cycle_stats_short, cycle_stats_long)

In [10]:
df, cycle_stats_short, cycle_stats_long = cycle_tracker(df, 0.03, 7) # sensitivity is adjustable | 0.0 would find every change of price direction

In [11]:
peaks_troughs =  df.index[df['peak_trough'] == 'trough'].tolist()
peaks_troughs2 =  df.index[df['peak_trough'] == 'peak'].tolist()
peaks_troughs.extend(peaks_troughs2)
df2 = df.loc[peaks_troughs, ['date', 'value']]

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = df['date'],
    y = df['value'],
    mode='lines+markers',
    name='Original Plot'
))

fig.add_trace(go.Scatter(
    x = df2['date'],
    y = df2['value'],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks and Troughs'
))


fig.show()

In [14]:
cycle_stats_short.head()

,full_cycle_number,number_days,mean,std,min,25%,50%,75%,max,range_width
0,1,8.0,135.800000,8.882246,126.7,128.550,132.65,145.600,146.2,19.5
1,2,16.0,140.306250,5.455086,134.1,136.425,137.85,144.375,150.9,16.8
2,3,25.0,137.520000,7.148543,128.0,130.800,136.40,144.000,149.5,21.5
3,4,10.0,136.990000,8.956494,127.7,129.025,134.60,145.850,149.3,21.6
4,5,33.0,146.527273,5.627070,140.2,142.100,146.30,148.800,160.0,19.8


In [7]:
cycle_stats_long.head()

,long_term_cycle_identifier,number_days,mean,std,min,25%,50%,75%,max,range_width
0,1,743.0,133.400673,11.664172,115.5,124.5,127.9,139.5,162.7,47.2
1,2,478.0,146.638703,4.885093,135.6,144.8,146.5,147.7,162.6,27.0


In [15]:
df.head()

,date,value,movement,movement_perc,direction,peak_trough,peak_trough_strength_perc,absolute_change_since_last_peak_trough,peak_trough_days,full_cycle_number,...,current_price_cycle_max_difference,short_cycle_min,short_cycle_max,long_term_cycle_identifier,long_cycle_absolute_change,long_term_cycle_days,long_term_cycle_difference_min,long_term_cycle_difference_max,long_term_cycle_min,long_term_cycle_max
4884,9/10/2016,128.8,0.0,0.000000,steady,mid-cycle,0.000000,0.0,0,1,...,0.0,128.8,128.8,1,0.0,0,0.0,0.0,128.8,128.8
4885,10/10/2016,127.8,-1.0,-0.007764,down,mid-cycle,-0.007764,-1.0,1,1,...,-1.0,127.8,128.8,1,-1.0,1,0.0,-1.0,127.8,128.8
4886,11/10/2016,126.7,-1.1,-0.008607,down,mid-cycle,-0.016304,-2.1,2,1,...,-2.1,126.7,128.8,1,-2.1,2,0.0,-2.1,126.7,128.8
4887,12/10/2016,128.9,2.2,0.017364,up,mid-cycle,0.000776,0.1,3,1,...,0.0,126.7,128.9,1,0.1,3,2.2,0.0,126.7,128.9
4888,13/10/2016,136.4,7.5,0.058185,up,mid-cycle,0.059006,7.6,4,1,...,0.0,126.7,136.4,1,7.6,4,9.7,0.0,126.7,136.4
